# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

### Importer les données

In [167]:
df_categories = pd.read_json("./data/US_category_id.json")
df_categories["id"] = df_categories["items"].apply(lambda x : int(x.get("id")))

df_o = df_youtube.merge(df_categories, right_on = 'id', left_on = "category_id")
df_o.rename({"video_id": "_id"}, axis=1, inplace=True)

In [168]:
df_o_w_d = df_o.drop_duplicates("_id")

In [169]:
df_o._id.shape, df_o._id.unique().shape

((7992,), (2364,))

In [171]:
collection.insert_many(df_o_w_d.to_dict(orient = 'records'))
collection.find_one()

{'_id': 'XpVt6Z1Gjjo',
 'category_id': 24,
 'channel_title': 'Logan Paul Vlogs',
 'comment_total': 46245,
 'date': 13.09,
 'dislikes': 5931,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 24,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'id': '24',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Entertainment'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 320053,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'title': '1 YEAR OF VLOGGING --

In [170]:
collection.drop()

## Question 1  

In [172]:
import pprint

In [173]:
for video in collection.find({"channel_title" : "Apple"}):
    pprint.pprint(video)

{'_id': 'K4wEI5zhHB0',
 'category_id': 28,
 'channel_title': 'Apple',
 'comment_total': 0,
 'date': 13.09,
 'dislikes': 26679,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 28,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
           'id': '28',
           'kind': 'youtube#videoCategory',
           'snippet': {'assignable': True,
                       'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                       'title': 'Science & Technology'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 185853,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 '
         'Bionic|augmented reality|emoji|animoji|Face ID|Apple '
         'Pay|camera|smartphone',
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'views': 7860119}
{'_id': 'A9k88sMyiJM',
 'category_id': 28,
 'channel_title': 'Apple',
 'comment_total': 0,
 'date': 18.09,
 

## Question 2

In [94]:
for a in collection.aggregate([{"$group" : { "_id" : "$category_id", "videosNumberByCategory" : { "$sum" : 1}}}]):
    pprint.pprint(a)

{'_id': 19, 'videosNumberByCategory': 384}
{'_id': 29, 'videosNumberByCategory': 112}
{'_id': 15, 'videosNumberByCategory': 928}
{'_id': 20, 'videosNumberByCategory': 656}
{'_id': 23, 'videosNumberByCategory': 6040}
{'_id': 22, 'videosNumberByCategory': 7056}
{'_id': 25, 'videosNumberByCategory': 4984}
{'_id': 17, 'videosNumberByCategory': 3280}
{'_id': 2, 'videosNumberByCategory': 928}
{'_id': 26, 'videosNumberByCategory': 6952}
{'_id': 27, 'videosNumberByCategory': 2672}
{'_id': 43, 'videosNumberByCategory': 16}
{'_id': 1, 'videosNumberByCategory': 3024}
{'_id': 24, 'videosNumberByCategory': 12808}
{'_id': 28, 'videosNumberByCategory': 4096}
{'_id': 10, 'videosNumberByCategory': 10000}


## Question 3

## Question 4

## Question 5

## Question 6 